# Volume 1, Chapter 9: Working with Network Data

**Parse Multi-Vendor Configs with TextFSM and AI**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/main/Volume-1-Foundations/Colab-Notebooks/Vol1_Ch9_Network_Data.ipynb)

---

**What you'll learn:**
- 📋 Parse CLI output with TextFSM
- 🔄 Handle multi-vendor differences
- 🤖 Use AI for flexible parsing
- 📊 Normalize data across vendors

In [ ]:
!pip install -q anthropic ntc-templates textfsm

import os
from getpass import getpass

try:
    from google.colab import userdata
    os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
except:
    if 'ANTHROPIC_API_KEY' not in os.environ:
        os.environ['ANTHROPIC_API_KEY'] = getpass('Anthropic API key: ')

from anthropic import Anthropic
client = Anthropic()
print("✓ Ready!")

---
## 📋 Example 1: Parse with NTC Templates

In [ ]:
from ntc_templates.parse import parse_output

# Cisco IOS "show ip interface brief" output
ios_output = """Interface              IP-Address      OK? Method Status                Protocol
GigabitEthernet0/0     192.168.1.1     YES manual up                    up
GigabitEthernet0/1     10.0.0.1        YES NVRAM  up                    up
Loopback0              1.1.1.1         YES manual up                    up
GigabitEthernet0/2     unassigned      YES unset  administratively down down"""

parsed = parse_output(
    platform="cisco_ios",
    command="show ip interface brief",
    data=ios_output
)

print("📋 NTC TEMPLATES PARSING")
print("=" * 60)
print(f"Parsed {len(parsed)} interfaces:\n")

for iface in parsed:
    print(f"  {iface['intf']:25} {iface['ipaddr']:15} {iface['status']}")

---
## 🔄 Example 2: Multi-Vendor Challenge

In [ ]:
# Same info, different vendors
cisco_output = """Interface              IP-Address      OK? Method Status                Protocol
GigabitEthernet0/0     192.168.1.1     YES manual up                    up"""

juniper_output = """Interface       Admin Link Proto    Local
ge-0/0/0        up    up
ge-0/0/0.0      up    up   inet     192.168.1.1/24"""

arista_output = """Interface       IP Address          Status     Protocol
Ethernet1       192.168.1.1/24      up         up"""

print("🔄 MULTI-VENDOR OUTPUTS (Same Data, Different Format)")
print("=" * 60)
print("\nCisco IOS:")
print(cisco_output)
print("\nJuniper:")
print(juniper_output)
print("\nArista:")
print(arista_output)

---
## 🤖 Example 3: AI-Powered Universal Parser

In [ ]:
import json
import re

def ai_parse_interfaces(output, vendor="unknown"):
    """Parse any vendor's interface output using AI."""
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=500,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Parse this {vendor} CLI output into JSON.

Output:
{output}

Return JSON array with objects containing:
- interface: normalized name (e.g., "Gi0/0" or "ge-0/0/0")
- ip_address: IP (or null if none)
- status: "up" or "down"

Return ONLY valid JSON, no explanation."""
        }]
    )
    
    # Extract JSON from response
    text = response.content[0].text
    json_match = re.search(r'\[.*\]', text, re.DOTALL)
    if json_match:
        return json.loads(json_match.group())
    return []

print("🤖 AI UNIVERSAL PARSER")
print("=" * 60)

vendors = [
    ("Cisco IOS", cisco_output),
    ("Juniper", juniper_output),
    ("Arista", arista_output),
]

for vendor, output in vendors:
    parsed = ai_parse_interfaces(output, vendor)
    print(f"\n{vendor}:")
    for iface in parsed:
        print(f"  {iface}")

---
## 📊 Example 4: Normalize to Common Schema

In [ ]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class NormalizedInterface:
    """Vendor-agnostic interface representation."""
    name: str
    short_name: str
    ip_address: Optional[str]
    subnet_mask: Optional[str]
    status: str
    vendor: str

def normalize_interface_name(name, vendor):
    """Convert vendor-specific names to standard format."""
    mapping = {
        "GigabitEthernet": "Gi",
        "FastEthernet": "Fa",
        "Ethernet": "Eth",
        "ge-": "ge-",
        "xe-": "xe-",
    }
    
    short = name
    for full, abbrev in mapping.items():
        if name.startswith(full):
            short = name.replace(full, abbrev, 1)
            break
    
    return short

def ai_normalize(output, vendor):
    """Parse and normalize in one step."""
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=500,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Parse this {vendor} output to normalized JSON.

{output}

Return JSON array:
[
  {{
    "name": "full interface name",
    "short_name": "abbreviated (Gi0/0, ge-0/0/0, Eth1)",
    "ip_address": "x.x.x.x or null",
    "subnet_mask": "x.x.x.x or null",
    "status": "up or down",
    "vendor": "{vendor.lower()}"
  }}
]

ONLY JSON, no text."""
        }]
    )
    
    text = response.content[0].text
    json_match = re.search(r'\[.*\]', text, re.DOTALL)
    return json.loads(json_match.group()) if json_match else []

print("📊 NORMALIZED OUTPUT")
print("=" * 60)

all_interfaces = []
for vendor, output in vendors:
    normalized = ai_normalize(output, vendor)
    all_interfaces.extend(normalized)

print("\nAll interfaces (normalized):")
for iface in all_interfaces:
    print(f"  {iface['short_name']:15} {iface['ip_address'] or 'N/A':15} {iface['status']:5} ({iface['vendor']})")

---
## 📝 Example 5: Parse Syslog with AI

In [ ]:
logs = [
    "Jan 15 10:23:45 CORE-RTR %OSPF-5-ADJCHG: Process 1, Nbr 10.1.1.2 on Gi0/0 from FULL to DOWN",
    "Jan 15 10:23:46 CORE-RTR %LINK-3-UPDOWN: Interface GigabitEthernet0/0, changed state to down",
    "Jan 15 10:25:00 CORE-RTR %SYS-5-CONFIG_I: Configured from console by admin on vty0",
]

def parse_syslog(log):
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=200,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Parse this syslog to JSON:
{log}

Return: {{"timestamp": "...", "device": "...", "facility": "...", "severity": 0-7, "message": "..."}}
ONLY JSON."""
        }]
    )
    text = response.content[0].text
    json_match = re.search(r'\{.*\}', text, re.DOTALL)
    return json.loads(json_match.group()) if json_match else {}

print("📝 SYSLOG PARSING")
print("=" * 60)

for log in logs:
    parsed = parse_syslog(log)
    print(f"\n{parsed.get('device', 'Unknown')} | Sev:{parsed.get('severity', '?')} | {parsed.get('facility', 'Unknown')}")
    print(f"  → {parsed.get('message', log)[:60]}")

---
## 🎯 Key Takeaways

| Approach | Best For | Pros | Cons |
|----------|----------|------|------|
| NTC Templates | Standard commands | Fast, no API cost | Limited to known formats |
| AI Parsing | Any format | Flexible, handles variations | API cost, slower |
| Hybrid | Production | Best of both | More complex |

**Recommendation:**
1. Try NTC Templates first (free, fast)
2. Fall back to AI for unknown formats
3. Normalize everything to common schema

---

## 📚 Next Steps

➡️ [Chapter 10: API Integration](./Vol1_Ch10_API_Integration.ipynb)